In [3]:
import os
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()
training_file_name = "attack_pattern_training2.jsonl"
print('Done')#%%


Done


In [8]:
from src.model.container import AttackPatternsContainer
import json
from src.model.interfaceToMitre.mitreData.utils.FileUtils import write_to_file, read_from_file

messages = []

for at in [at for at in AttackPatternsContainer().get_data() if 'atlas' in at.x_mitre_domains]:
    if 'AML' in at.x_mitre_id:
        system_targeted = ['LLM model']
    else:
        system_targeted = at.x_mitre_platforms

    json_data = {
                "role": "system",
                "content": "You are a chatbot specializing in the context of cybersecurity, the attack pattern known with the id \'" + at.x_mitre_id + "\' is: " + at.name + ' .'
    }

    messages.append(json_data)

    
print('Done')

Done


In [14]:
load_dotenv()

print(os.getenv("AZURE_OPENAI_API_KEY"), os.getenv("AZURE_OPENAI_ENDPOINT"))

client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-02-01",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

deployment_name = 'gpt-fine-tuning-test'
# This will correspond to the custom name you chose for your deployment when you deployed a model. Use a gpt-35-turbo-instruct deployment.

messages.append({
            "role": "user",
            "content": "The description of these CVE to what attack pattern is related to? Description: TensorFlow is an Open Source Machine Learning Framework. In versions prior to 2.11.1 a malicious invalid input crashes a tensorflow model (Check Failed) and can be used to trigger a denial of service attack. A proof of concept can be constructed with the `Convolution3DTranspose` function. This Convolution3DTranspose layer is a very common API in modern neural networks. The ML models containing such vulnerable components could be deployed in ML applications or as cloud services. This failure could be potentially used to trigger a denial of service attack on ML cloud services. An attacker must have privilege to provide input to a `Convolution3DTranspose` call. This issue has been patched and users are advised to upgrade to version 2.11.1. There are no known workarounds for this vulnerability. Return the output in the format: id: <attack_pattern_id>."
        })

response = client.chat.completions.create(
    model=deployment_name,
    messages=messages
)

print(response.choices[0].message.content)

a76fd807e235483c86ee29e522b499f9 https://cti-gpt-fine-tuned.openai.azure.com/
HTTP Request: POST https://cti-gpt-fine-tuned.openai.azure.com//openai/deployments/gpt-fine-tuning-test/chat/completions?api-version=2024-02-01 "HTTP/1.1 200 OK"
Based on the description provided, the attack pattern related to the CVE described is:

id: `AML.T0029`

This attack pattern involves "Denial of ML Service," where a malicious input can crash a TensorFlow model, resulting in a denial of service attack.


In [11]:
print(AttackPatternsContainer().get_object_from_data_by_mitre_id('AML.T0057').name)

LLM Data Leakage
